# MySQL database

This notebook will successfully connect to the MySQL database for the MSDS 420 to benchmark for the NYC Congestion Relief Zone policy analysis.

In [1]:
# Import libraries
!pip install mysql-connector-python
!pip install great_tables
from great_tables import GT, md, html
import mysql.connector
import pandas as pd
import numpy as np
import time

In [2]:
# Load datasets
# Daily Ridership dataset
ridership_df = pd.read_csv('UPDATED MTA_Daily_Ridership_and_Traffic__Beginning_2020_20250823.csv',
                           usecols = ['Date', 'Mode', 'Count']
                          )
ridership_df.columns = ridership_df.columns.str.lower() # Make columns easier to read
ridership_df['date'] = pd.to_datetime(ridership_df['date'], errors='coerce') # Ensure datetime format
ridership_df = ridership_df[
    ~((ridership_df['date'].dt.year == 2025) & 
      (ridership_df['date'].dt.month == 8))
    ] # Remove August entries (incomplete month)
ridership_df = ridership_df[ridership_df['count'].notnull()] # Remove nulls
ridership_df['date'] = ridership_df['date'].dt.strftime('%Y-%m-%d') # Convert format
print(ridership_df.head())

           date    mode      count
147  2025-07-31     Bus  1310912.0
148  2025-07-31  Subway  3718979.0
149  2025-07-31     SIR     7004.0
150  2025-07-31     AAR    42155.0
151  2025-07-31     MNR   229384.0


In [3]:
# Bridges and Tunnels dataset
bridges_tunnels_df = pd.read_csv('UPDATED MTA_Bridges_and_Tunnels_Hourly_Crossings__Beginning_2019_20250823.csv',
                                 usecols = ['Date', 'Direction', 'Traffic Count']
                                )
bridges_tunnels_df.columns = bridges_tunnels_df.columns.str.lower().str.replace(' ', '_') # Make columns easier to read
bridges_tunnels_df['date'] = pd.to_datetime(bridges_tunnels_df['date'], errors='coerce') # Ensure datetime format
bridges_tunnels_df = bridges_tunnels_df[
    ~((bridges_tunnels_df['date'].dt.year == 2025) & 
      (bridges_tunnels_df['date'].dt.month == 8))
    ] # Remove August entries (incomplete month)
bridges_tunnels_df['date'] = bridges_tunnels_df['date'].dt.strftime('%Y-%m-%d') # Convert format
print(bridges_tunnels_df.head())

         date                          direction  traffic_count
0  2025-07-29         Westbound to Staten Island             22
1  2025-07-29  Southbound to Manhattan or Queens              3
2  2025-07-29  Southbound to Manhattan or Queens             28
3  2025-07-29   Northbound to Manhattan or Bronx              2
4  2025-07-29         Westbound to Staten Island              6


In [4]:
# Congestion Relief Zone dataset
crz_df = pd.read_csv('UPDATED MTA_Congestion_Relief_Zone_Vehicle_Entries__Beginning_2025_20250823.csv',
                    usecols = ['Toll Date', 'Time Period', 'Vehicle Class', 'CRZ Entries']
                    )
crz_df.columns = crz_df.columns.str.lower().str.replace(' ', '_') # Make columns easier to read
crz_df['toll_date'] = pd.to_datetime(crz_df['toll_date'], errors='coerce') # Ensure datetime format
crz_df = crz_df[crz_df['toll_date'].dt.month != 8] # Remove August entries (incomplete month)
crz_df['toll_date'] = crz_df['toll_date'].dt.strftime('%Y-%m-%d') # Convert format
print(crz_df.head())

         toll_date time_period               vehicle_class  crz_entries
165888  2025-07-31   Overnight  1 - Cars, Pickups and Vans           73
165889  2025-07-31   Overnight                TLC Taxi/FHV           54
165890  2025-07-31   Overnight                TLC Taxi/FHV          131
165891  2025-07-31   Overnight                TLC Taxi/FHV           61
165892  2025-07-31   Overnight                TLC Taxi/FHV          118


In [5]:
# Run docker file
!docker compose -f docker-compose-mysql.yml up -d

 Container mysql-db  Running


In [6]:
# Connect to MySQL

# Establish connection
mydb = mysql.connector.connect(
    host="127.0.0.1",
    port=3307,
    user="root",
    password="",         # empty password
    database="benchmarkdb",
    allow_local_infile=True
)
    
mycursor = mydb.cursor() # Create cursor

In [7]:
# Drop table if it exists
table_name = "bridges_tunnels_data"
mycursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    
# Create table
sql_create_table = f"""
    CREATE TABLE IF NOT EXISTS `{table_name}` (
        date DATE,
        direction VARCHAR(50),
        traffic_count INT
        );
        """
    
mycursor.execute(sql_create_table) # Excute table query
mydb.commit() # Commit changes
print(f"Table `{table_name}` created successfully!\n")

# Insert data into table
insert_query = """
INSERT INTO bridges_tunnels_data (
    date, 
    direction,
    traffic_count
) VALUES (%s, %s, %s)
"""

# Convert DataFrame to list of tuples
data_tuples = list(bridges_tunnels_df[
    ['date', 'direction', 'traffic_count']
    ].itertuples(index=False, name=None))

# Time loading/inserting
start_time = time.time()

# Batch insert
batch_size = 150000
for start in range(0, len(data_tuples), batch_size):
    batch = data_tuples[start:start + batch_size]
    mycursor.executemany(insert_query, batch)
    mydb.commit() # Commit changes
    print(f"Inserted rows {start} to {start + len(batch) - 1}")

# End loading
end_time = time.time()
elapsed_time_1 = end_time - start_time

print("\nBridge & Tunnel data inserted successfully!")
print(f"\nTotal Loading Time for Bridges and & Tunnel data table: {elapsed_time_1} seconds")

Table `bridges_tunnels_data` created successfully!

Inserted rows 0 to 149999
Inserted rows 150000 to 299999
Inserted rows 300000 to 449999
Inserted rows 450000 to 599999
Inserted rows 600000 to 749999
Inserted rows 750000 to 899999
Inserted rows 900000 to 1049999
Inserted rows 1050000 to 1199999
Inserted rows 1200000 to 1349999
Inserted rows 1350000 to 1499999
Inserted rows 1500000 to 1649999
Inserted rows 1650000 to 1799999
Inserted rows 1800000 to 1949999
Inserted rows 1950000 to 2099999
Inserted rows 2100000 to 2249999
Inserted rows 2250000 to 2399999
Inserted rows 2400000 to 2549999
Inserted rows 2550000 to 2699999
Inserted rows 2700000 to 2849999
Inserted rows 2850000 to 2999999
Inserted rows 3000000 to 3149999
Inserted rows 3150000 to 3299999
Inserted rows 3300000 to 3449999
Inserted rows 3450000 to 3599999
Inserted rows 3600000 to 3749999
Inserted rows 3750000 to 3899999
Inserted rows 3900000 to 4049999
Inserted rows 4050000 to 4199999
Inserted rows 4200000 to 4349999
Inserted 

In [8]:
# Drop table if it exists
table_name = "ridership_data"
mycursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    
# Create table
sql_create_table = f"""
    CREATE TABLE IF NOT EXISTS `{table_name}` (
        date DATE,
        mode VARCHAR(20),
        count INT
        );
        """
    
mycursor.execute(sql_create_table) # Excute table query
mydb.commit() # Commit changes
print(f"Table `{table_name}` created successfully!\n")

# Insert data into table
insert_query = """
INSERT INTO ridership_data (
    date, 
    mode,
    count
) VALUES (%s, %s, %s)
"""

# Convert DataFrame to list of tuples
data_tuples = list(ridership_df[
    ['date', 'mode', 'count']
    ].itertuples(index=False, name=None))

# Time loading/inserting
start_time = time.time()

# Batch insert
batch_size = 1000
for start in range(0, len(data_tuples), batch_size):
    batch = data_tuples[start:start + batch_size]
    mycursor.executemany(insert_query, batch)
    mydb.commit() # Commit changes
    print(f"Inserted rows {start} to {start + len(batch) - 1}")

# End loading/inserting
end_time = time.time()
elapsed_time_2 = end_time - start_time

print("\nRidership data inserted successfully!")
print(f"\nTotal Loading Time for Ridership data table: {elapsed_time_2} seconds")

Table `ridership_data` created successfully!

Inserted rows 0 to 999
Inserted rows 1000 to 1999
Inserted rows 2000 to 2999
Inserted rows 3000 to 3999
Inserted rows 4000 to 4999
Inserted rows 5000 to 5999
Inserted rows 6000 to 6999
Inserted rows 7000 to 7999
Inserted rows 8000 to 8999
Inserted rows 9000 to 9999
Inserted rows 10000 to 10999
Inserted rows 11000 to 11999
Inserted rows 12000 to 12999
Inserted rows 13000 to 13999
Inserted rows 14000 to 14086

Ridership data inserted successfully!

Total Loading Time for Ridership data table: 0.9410388469696045 seconds


In [9]:
# Drop table if it exists
table_name = "crz_data"
mycursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    
# Create table
sql_create_table = f"""
    CREATE TABLE IF NOT EXISTS `{table_name}` (
        toll_date DATE,
        time_period VARCHAR(20),
        vehicle_class VARCHAR(50),
        crz_entries INT
        );
        """
    
mycursor.execute(sql_create_table) # Excute table query
mydb.commit() # Commit changes
print(f"Table `{table_name}` created successfully!\n")

# Insert data into table
insert_query = """
INSERT INTO crz_data (
    toll_date, 
    time_period,
    vehicle_class,
    crz_entries
) VALUES (%s, %s, %s, %s)
"""

# Convert DataFrame to list of tuples
data_tuples = list(crz_df[
    ['toll_date', 'time_period', 'vehicle_class', 'crz_entries']
    ].itertuples(index=False, name=None))

# Time loading/inserting
start_time = time.time()

# Batch insert
batch_size = 150000
for start in range(0, len(data_tuples), batch_size):
    batch = data_tuples[start:start + batch_size]
    mycursor.executemany(insert_query, batch)
    mydb.commit() # Commit changes
    print(f"Inserted rows {start} to {start + len(batch) - 1}")

# End loading/inserting
end_time = time.time()
elapsed_time_3 = end_time - start_time

print("\nCRZ Entry data inserted successfully!")
print(f"\nTotal Indexing Time for CRZ data table: {elapsed_time_3} seconds")

Table `crz_data` created successfully!

Inserted rows 0 to 149999
Inserted rows 150000 to 299999
Inserted rows 300000 to 449999
Inserted rows 450000 to 599999
Inserted rows 600000 to 749999
Inserted rows 750000 to 899999
Inserted rows 900000 to 1049999
Inserted rows 1050000 to 1199999
Inserted rows 1200000 to 1349999
Inserted rows 1350000 to 1499999
Inserted rows 1500000 to 1649999
Inserted rows 1650000 to 1799999
Inserted rows 1800000 to 1949999
Inserted rows 1950000 to 2099999
Inserted rows 2100000 to 2156543

CRZ Entry data inserted successfully!

Total Indexing Time for CRZ data table: 26.533424615859985 seconds


In [10]:
# Define queries
revenue_case = """
CASE 
    WHEN vehicle_class IN ('1 - Cars, Pickups and Vans') AND time_period = 'Peak' THEN 9.00
    WHEN vehicle_class IN ('1 - Cars, Pickups and Vans') AND time_period = 'Overnight' THEN 2.25
    WHEN vehicle_class IN ('5 - Motorcycles') AND time_period = 'Peak' THEN 4.50
    WHEN vehicle_class IN ('5 - Motorcycles') AND time_period = 'Overnight' THEN 1.05
    WHEN vehicle_class IN ('2 - Single-Unit Trucks', '4 - Buses') AND time_period = 'Peak' THEN 14.40
    WHEN vehicle_class IN ('2 - Single-Unit Trucks', '4 - Buses') AND time_period = 'Overnight' THEN 3.60
    WHEN vehicle_class = '3 - Multi-Unit Trucks' AND time_period = 'Peak' THEN 21.60
    WHEN vehicle_class = '3 - Multi-Unit Trucks' AND time_period = 'Overnight' THEN 5.40
    WHEN vehicle_class = 'TLC Taxi/FHV' THEN 1.125
    ELSE 0
END
"""

queries = {
    "Pre-toll Average Daily Traffic (5/1-12/31)": """
        SELECT Year(date) AS year, 
               AVG(traffic_count) AS average_daily_traffic
        FROM bridges_tunnels_data
        WHERE DATE_FORMAT(date, '%m-%d') BETWEEN '05-01' AND '12-31'
            AND direction LIKE '%to Manhattan%'
        GROUP BY year
        ORDER BY year
    """,
     "Post-toll Average Daily Traffic (1/5-7/31)": """
        SELECT Year(date) AS year,
               AVG(traffic_count) AS average_daily_traffic
        FROM bridges_tunnels_data
        WHERE DATE_FORMAT(date, '%m-%d') BETWEEN '01-05' AND '07-31'
            AND direction LIKE '%to Manhattan%'
        GROUP BY year
        ORDER BY year
    """,
    "Average Ridership By Mode (Jan-July)": """
        SELECT mode,
           AVG(count) AS average_monthly_ridership
        FROM ridership_data
        WHERE YEAR(date) = '2025'
          AND MONTH(date) BETWEEN 1 AND 7
        GROUP BY mode
        ORDER BY mode;
    """,
    "Average Subway Yearly Ridership (Jan–Jul)": """
        SELECT YEAR(date) AS year,
               AVG(count) AS average_subway_ridership
        FROM ridership_data
        WHERE mode = 'Subway'
            AND MONTH(date) <= 7
        GROUP BY year
        ORDER BY year;
    """,
    "Average Subway Post-Toll Monthly Ridership": """
        SELECT DATE_FORMAT(date, '%Y-%m') AS month, 
               AVG(count) as average_subway_ridership
        FROM ridership_data
        WHERE mode = 'Subway' 
            AND YEAR(date) = '2025'
        GROUP BY month
        ORDER BY month;
    """,
    "Monthly Estimated Revenue": f"""
        SELECT DATE_FORMAT(toll_date, '%Y-%m') AS month, 
               SUM(crz_entries * ({revenue_case})) AS est_revenue
        FROM crz_data
        GROUP BY month
        ORDER BY month;
    """,
    "Estimated Revenue By Vehicle Class": f"""
        SELECT vehicle_class, 
               SUM(crz_entries * ({revenue_case})) AS est_revenue
        FROM crz_data
        GROUP BY vehicle_class
        ORDER BY vehicle_class, est_revenue DESC;
    """
}

# Run benchmarks
results = []
dfs = {}
for name, query in queries.items():
    start_time = time.time()
    df = pd.read_sql_query(query, mydb)
    elapsed = time.time() - start_time
    results.append((name, round(elapsed, 4)))
    dfs[name] = df

# Add total time as a final row
total_time = elapsed_time_1 + elapsed_time_2 + elapsed_time_3
results.append(("TOTAL INSERT TIME (All Tables)", round(total_time, 4)))
results_df = pd.DataFrame(results, columns=["Query", "Time (seconds)"])
results_df["Query"] = results_df["Query"].str.strip() # Clean up Query names by stripping leading/trailing spaces

print("MySQL Benchmark Results (seconds):")
print(results_df.to_string(index=False, justify="left"))

C:\Users\joshp\AppData\Local\Temp\ipykernel_41088\912555425.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, mydb)


MySQL Benchmark Results (seconds):
Query                                       Time (seconds)
Pre-toll Average Daily Traffic (5/1-12/31)   9.7453       
Post-toll Average Daily Traffic (1/5-7/31)   9.6641       
      Average Ridership By Mode (Jan-July)   0.0115       
 Average Subway Yearly Ridership (Jan–Jul)   0.0104       
Average Subway Post-Toll Monthly Ridership   0.0106       
                 Monthly Estimated Revenue   4.5285       
        Estimated Revenue By Vehicle Class   3.5786       
            TOTAL INSERT TIME (All Tables) 163.3907       


In [11]:
daily_traffic_pre_tax_df = dfs["Pre-toll Average Daily Traffic (5/1-12/31)"]
daily_traffic_post_tax_df = dfs["Post-toll Average Daily Traffic (1/5-7/31)"]
average_mode_ridership_df = dfs["Average Ridership By Mode (Jan-July)"]
subway_yearly_ridership_df = dfs["Average Subway Yearly Ridership (Jan–Jul)"]
subway_monthly_ridership_df = dfs["Average Subway Post-Toll Monthly Ridership"]
monthly_revenue_df = dfs["Monthly Estimated Revenue"]
revenue_vehicle_class_df = dfs["Estimated Revenue By Vehicle Class"]

# Pretty print
gt_table1 = (
    GT(daily_traffic_pre_tax_df)
    .tab_header(title=html("Pre-toll Average Daily Traffic (5/1-12/31)"))
    .cols_label(
        year="Year", 
        average_daily_traffic="Average Daily Traffic"
    )
)
gt_table1

GT(_tbl_data=   year  average_daily_traffic
0  2019               199.4982
1  2020               153.5634
2  2021               199.8586
3  2022               207.1922
4  2023               211.6507
5  2024               213.2201
6  2025               216.1633, _body=<great_tables._gt_data.Body object at 0x00000242B0809550>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='Year', column_align='right', column_width=None), ColInfo(var='average_daily_traffic', type=<ColInfoTypeEnum.default: 1>, column_label='Average Daily Traffic', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000242B08092B0>, _spanners=Spanners([]), _heading=Heading(title=Html(text='Pre-toll Average Daily Traffic (5/1-12/31)'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000242B0809940>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, category='heading', type='px', value='125%'), heading_title_font_weight=O

In [12]:
gt_table2 = (
    GT(daily_traffic_post_tax_df)
    .tab_header(title=html("Post-toll Average Daily Traffic (1/5-7/31)"))
    .cols_label(
        year="Year", 
        average_daily_traffic="Average Daily Traffic"
    )
)
gt_table2

GT(_tbl_data=   year  average_daily_traffic
0  2019               202.8286
1  2020               143.2220
2  2021               183.8750
3  2022               204.1729
4  2023               208.4227
5  2024               211.4927
6  2025               210.6497, _body=<great_tables._gt_data.Body object at 0x00000242B077AEA0>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='Year', column_align='right', column_width=None), ColInfo(var='average_daily_traffic', type=<ColInfoTypeEnum.default: 1>, column_label='Average Daily Traffic', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000242B038FD90>, _spanners=Spanners([]), _heading=Heading(title=Html(text='Post-toll Average Daily Traffic (1/5-7/31)'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000242B0870410>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, category='heading', type='px', value='125%'), heading_title_font_weight=O

In [13]:
gt_table3 = (
    GT(average_mode_ridership_df)
    .tab_header(title=html("Average Ridership By Mode (Jan–Jul 2025)"))
    .cols_label(
        mode="Mode", 
        average_monthly_ridership= "Average Ridership"
    )
)
gt_table3

GT(_tbl_data=          mode  average_monthly_ridership
0          AAR               3.623963e+04
1           BT               9.198175e+05
2          Bus               1.229608e+06
3  CBD Entries               5.563550e+05
4  CRZ Entries               4.912280e+05
5         LIRR               2.164496e+05
6          MNR               1.851457e+05
7          SIR               5.781259e+03
8       Subway               3.473793e+06, _body=<great_tables._gt_data.Body object at 0x00000242B07CF350>, _boxhead=Boxhead([ColInfo(var='mode', type=<ColInfoTypeEnum.default: 1>, column_label='Mode', column_align='left', column_width=None), ColInfo(var='average_monthly_ridership', type=<ColInfoTypeEnum.default: 1>, column_label='Average Ridership', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000242B08702D0>, _spanners=Spanners([]), _heading=Heading(title=Html(text='Average Ridership By Mode (Jan–Jul 2025)'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000242B08707D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, ca

In [14]:
gt_table4 = (
    GT(subway_yearly_ridership_df)
    .tab_header(title=html("Average Subway Yearly Ridership (Jan–Jul)"))
    .cols_label(
        year="Year", 
        average_subway_ridership= "Average Subway Ridership"
    )
)
gt_table4

GT(_tbl_data=   year  average_subway_ridership
0  2020              1.026594e+06
1  2021              1.769227e+06
2  2022              2.644680e+06
3  2023              3.118955e+06
4  2024              3.185480e+06
5  2025              3.473793e+06, _body=<great_tables._gt_data.Body object at 0x00000242B07FAA50>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='Year', column_align='right', column_width=None), ColInfo(var='average_subway_ridership', type=<ColInfoTypeEnum.default: 1>, column_label='Average Subway Ridership', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000242B077B230>, _spanners=Spanners([]), _heading=Heading(title=Html(text='Average Subway Yearly Ridership (Jan–Jul)'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000242B077B100>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, category='heading', type='px', value='125%'), heading_title_font_weight=Option

In [15]:
gt_table5 = (
    GT(subway_monthly_ridership_df)
    .tab_header(title=html("Average Subway Monthly Ridership (Jan–Jul 2025)"))
    .cols_label(
        month="Month",  
        average_subway_ridership= "Average Subway Ridership"
    )
)
gt_table5

GT(_tbl_data=     month  average_subway_ridership
0  2025-01              3.188048e+06
1  2025-02              3.361130e+06
2  2025-03              3.494515e+06
3  2025-04              3.666954e+06
4  2025-05              3.627925e+06
5  2025-06              3.551575e+06
6  2025-07              3.424240e+06, _body=<great_tables._gt_data.Body object at 0x00000242B080E350>, _boxhead=Boxhead([ColInfo(var='month', type=<ColInfoTypeEnum.default: 1>, column_label='Month', column_align='right', column_width=None), ColInfo(var='average_subway_ridership', type=<ColInfoTypeEnum.default: 1>, column_label='Average Subway Ridership', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000242B077BA80>, _spanners=Spanners([]), _heading=Heading(title=Html(text='Average Subway Monthly Ridership (Jan–Jul 2025)'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000242B077B820>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, category='he

In [16]:
gt_table6 = (
    GT(monthly_revenue_df)
    .tab_header(title=html("Monthly Estimated Revenue"))
    .cols_label(
        month="Month", 
        est_revenue= "Estimated Revenue"
    )
    .fmt_currency(columns="est_revenue")
)
gt_table6

GT(_tbl_data=     month   est_revenue
0  2025-01  7.162038e+07
1  2025-02  7.382396e+07
2  2025-03  8.432042e+07
3  2025-04  8.369648e+07
4  2025-05  8.798363e+07
5  2025-06  8.350449e+07
6  2025-07  8.433866e+07, _body=<great_tables._gt_data.Body object at 0x00000242B025EE10>, _boxhead=Boxhead([ColInfo(var='month', type=<ColInfoTypeEnum.default: 1>, column_label='Month', column_align='right', column_width=None), ColInfo(var='est_revenue', type=<ColInfoTypeEnum.default: 1>, column_label='Estimated Revenue', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000242B085AD50>, _spanners=Spanners([]), _heading=Heading(title=Html(text='Monthly Estimated Revenue'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000242B085B1D0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x00000242B0809160>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, category='heading', type='px', value='125%'), heading_title_font_weight=OptionsInfo(scs

In [17]:
gt_table7 = (
    GT(revenue_vehicle_class_df)
    .tab_header(title=html("Estimated Revenue By Vehicle Class"))
    .cols_label(
        vehicle_class="Vehicle Class", 
        est_revenue= "Estimated Revenue"
    )
    .fmt_currency(columns="est_revenue")
)
gt_table7

GT(_tbl_data=                vehicle_class   est_revenue
0  1 - Cars, Pickups and Vans  4.497279e+08
1      2 - Single-Unit Trucks  4.811358e+07
2       3 - Multi-Unit Trucks  4.861388e+06
3                   4 - Buses  2.267271e+07
4             5 - Motorcycles  1.516298e+06
5                TLC Taxi/FHV  4.239614e+07, _body=<great_tables._gt_data.Body object at 0x000002429A376AF0>, _boxhead=Boxhead([ColInfo(var='vehicle_class', type=<ColInfoTypeEnum.default: 1>, column_label='Vehicle Class', column_align='left', column_width=None), ColInfo(var='est_revenue', type=<ColInfoTypeEnum.default: 1>, column_label='Estimated Revenue', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000242B07CFDF0>, _spanners=Spanners([]), _heading=Heading(title=Html(text='Estimated Revenue By Vehicle Class'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000242B07CFF00>, _formats=[<great_tables._gt_data.FormatInfo object at 0x00000242B0871BD0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), he

In [18]:
# Close connection
mycursor.close()
mydb.close()